In [1]:
import os

# please set cwd to the root of mmrazor repo.

# 使用MMRazor对ResNet34进行剪枝

本教程主要介绍如何手动配置剪枝config。

## 回顾MMCls

### 跨库调用resnet34配置文件

首先我们先跨库调用resnet34的配置文件。通过跨库调用，我们可以继承原有配置文件的所有内容。

In [2]:
# prepare work_dir
work_dir = './demo/tmp/'
if not os.path.exists(work_dir):
    os.mkdir(work_dir)

In [3]:
from mmengine import Config


def write_config(config_str, filename):
    with open(filename, 'w') as f:
        f.write(config_str)


In [4]:
# Prepare pretrain config
pretrain_config_path = f'{work_dir}/pretrain.py'
config_string = """
_base_ = ['mmcls::resnet/resnet34_8xb32_in1k.py']
"""
write_config(config_string, pretrain_config_path)
print(Config.fromfile(pretrain_config_path)['model'])

{'type': 'ImageClassifier', 'backbone': {'type': 'ResNet', 'depth': 34, 'num_stages': 4, 'out_indices': (3,), 'style': 'pytorch'}, 'neck': {'type': 'GlobalAveragePooling'}, 'head': {'type': 'LinearClsHead', 'num_classes': 1000, 'in_channels': 512, 'loss': {'type': 'CrossEntropyLoss', 'loss_weight': 1.0}, 'topk': (1, 5)}, '_scope_': 'mmcls'}


/home/liukai/miniconda3/envs/lab2max/lib/python3.9/site-packages/mmengine/config/utils.py:51: UserWarning: There is not `Config` define in {'Name': 'convnext-base_3rdparty_in21k', 'Metadata': {'Training Data': 'ImageNet-21k', 'FLOPs': 15359124480, 'Parameters': 88591464}, 'In Collection': 'ConvNeXt', 'Results': None, 'Weights': 'https://download.openmmlab.com/mmclassification/v0/convnext/convnext-base_3rdparty_in21k_20220124-13b83eec.pth', 'Converted From': {'Weights': 'https://dl.fbaipublicfiles.com/convnext/convnext_base_22k_224.pth', 'Code': 'https://github.com/facebookresearch/ConvNeXt'}}
  warnings.warn(f'There is not `Config` define in {model_cfg}')
/home/liukai/miniconda3/envs/lab2max/lib/python3.9/site-packages/mmengine/config/utils.py:51: UserWarning: There is not `Config` define in {'Name': 'convnext-large_3rdparty_in21k', 'Metadata': {'Training Data': 'ImageNet-21k', 'FLOPs': 34368026112, 'Parameters': 197767336}, 'In Collection': 'ConvNeXt', 'Results': None, 'Weights': 'htt

In [5]:
# Run config
! timeout 2 python ./tools/train.py $prune_config_path

## 准备剪枝config

1. 增加pretrained参数
2. 将resnet34模型装入剪枝算法wrapper中
3. 配置剪枝比例
4. 运行

### 1. 增加预训练参数
我们将原有的’model‘字段取出，命名为architecture，并且给archtecture增加init_cfg字段用来加载预训练模型参数。

In [6]:
checkpoint_path = 'https://download.openmmlab.com/mmclassification/v0/resnet/resnet34_8xb32_in1k_20210831-f257d4e6.pth'
prune_config_path = work_dir + 'prune.py'
config_string += """\n
data_preprocessor = {'type': 'mmcls.ClsDataPreprocessor'}
architecture = _base_.model
architecture.update({
    'init_cfg': {
        'type':
        'Pretrained',
        'checkpoint':
        'https://download.openmmlab.com/mmclassification/v0/resnet/resnet34_8xb32_in1k_20210831-f257d4e6.pth'  # noqa
    }
})
"""
write_config(config_string, prune_config_path)

### 2. 将resnet34模型装入剪枝算法wrapper中

我们将原有的model作为architecture放入到ItePruneAlgorithm算法中，并且将ItePruneAlgorithm作为新的model字段。

In [7]:
config_string += """
target_pruning_ratio={}
model = dict(
    _delete_=True,
    _scope_='mmrazor',
    type='ItePruneAlgorithm',
    architecture=architecture,
    mutator_cfg=dict(
        type='ChannelMutator',
        channel_unit_cfg=dict(
            type='L1MutableChannelUnit',
            default_args=dict(choice_mode='ratio'))),
    target_pruning_ratio=target_pruning_ratio,
    step_freq=1,
    prune_times=1,
)
"""
write_config(config_string, prune_config_path)

配置到这一步时，我们的config文件已经能够运行了。但是因为我们没有配置target_pruning_ratio，因此现在跑起来就和直接用原有config跑起来没有区别，接下来我们会介绍如何配置剪枝比例

In [8]:
! timeout 2 python ./tools/train.py $prune_config_path

### 4. 配置剪枝比例

我们的模型使用tracer解析模型，进而获得剪枝节点，为了方便用户配置剪枝节点比例，我们提供了一个获得剪枝节点剪枝比例配置的工具。通过该工具，我们可以方便地对剪枝比例进行配置。

In [9]:
ratio_template_path=work_dir+'prune_ratio_template.json'
! python ./tools/pruning/get_channel_units.py $pretrain_config_path --choice -o $ratio_template_path  &> /dev/null 2>&1
! cat $ratio_template_path
! rm $ratio_template_path

{
    "backbone.conv1_(0, 64)_64":1.0,
    "backbone.layer1.0.conv1_(0, 64)_64":1.0,
    "backbone.layer1.1.conv1_(0, 64)_64":1.0,
    "backbone.layer1.2.conv1_(0, 64)_64":1.0,
    "backbone.layer2.0.conv1_(0, 128)_128":1.0,
    "backbone.layer2.0.conv2_(0, 128)_128":1.0,
    "backbone.layer2.1.conv1_(0, 128)_128":1.0,
    "backbone.layer2.2.conv1_(0, 128)_128":1.0,
    "backbone.layer2.3.conv1_(0, 128)_128":1.0,
    "backbone.layer3.0.conv1_(0, 256)_256":1.0,
    "backbone.layer3.0.conv2_(0, 256)_256":1.0,
    "backbone.layer3.1.conv1_(0, 256)_256":1.0,
    "backbone.layer3.2.conv1_(0, 256)_256":1.0,
    "backbone.layer3.3.conv1_(0, 256)_256":1.0,
    "backbone.layer3.4.conv1_(0, 256)_256":1.0,
    "backbone.layer3.5.conv1_(0, 256)_256":1.0,
    "backbone.layer4.0.conv1_(0, 512)_512":1.0,
    "backbone.layer4.0.conv2_(0, 512)_512":1.0,
    "backbone.layer4.1.conv1_(0, 512)_512":1.0,
    "backbone.layer4.2.conv1_(0, 512)_512":1.0
}

我们修改该配置模板如下，并且将替换到我们的剪枝配置文件中。

（该配置来源于：Li, Hao, et al. "Pruning filters for efficient convnets." arXiv preprint arXiv:1608.08710 (2016).）

In [10]:
target_config = """
un_prune = 1.0
stage_ratio_1 = 0.5
stage_ratio_2 = 0.4
stage_ratio_3 = 0.6
stage_ratio_4 = un_prune

target_pruning_ratio = {
    # stage 1
    'backbone.conv1_(0, 64)_64': un_prune,  # short cut layers
    'backbone.layer1.0.conv1_(0, 64)_64': stage_ratio_1,
    'backbone.layer1.1.conv1_(0, 64)_64': stage_ratio_1,
    'backbone.layer1.2.conv1_(0, 64)_64': un_prune,
    # stage 2
    'backbone.layer2.0.conv1_(0, 128)_128': un_prune,
    'backbone.layer2.0.conv2_(0, 128)_128': un_prune,  # short cut layers
    'backbone.layer2.1.conv1_(0, 128)_128': stage_ratio_2,
    'backbone.layer2.2.conv1_(0, 128)_128': stage_ratio_2,
    'backbone.layer2.3.conv1_(0, 128)_128': un_prune,
    # stage 3
    'backbone.layer3.0.conv1_(0, 256)_256': un_prune,
    'backbone.layer3.0.conv2_(0, 256)_256': un_prune,  # short cut layers
    'backbone.layer3.1.conv1_(0, 256)_256': stage_ratio_3,
    'backbone.layer3.2.conv1_(0, 256)_256': stage_ratio_3,
    'backbone.layer3.3.conv1_(0, 256)_256': stage_ratio_3,
    'backbone.layer3.4.conv1_(0, 256)_256': stage_ratio_3,
    'backbone.layer3.5.conv1_(0, 256)_256': un_prune,
    # stage 4
    'backbone.layer4.0.conv1_(0, 512)_512': stage_ratio_4,
    'backbone.layer4.0.conv2_(0, 512)_512': un_prune,  # short cut layers
    'backbone.layer4.1.conv1_(0, 512)_512': stage_ratio_4,
    'backbone.layer4.2.conv1_(0, 512)_512': stage_ratio_4
}
"""

In [11]:
config_string=config_string.replace('target_pruning_ratio={}',target_config)
write_config(config_string,prune_config_path)
! cat $prune_config_path


_base_ = ['mmcls::resnet/resnet34_8xb32_in1k.py']


data_preprocessor = {'type': 'mmcls.ClsDataPreprocessor'}
architecture = _base_.model
architecture.update({
    'init_cfg': {
        'type':
        'Pretrained',
        'checkpoint':
        'https://download.openmmlab.com/mmclassification/v0/resnet/resnet34_8xb32_in1k_20210831-f257d4e6.pth'  # noqa
    }
})


un_prune = 1.0
stage_ratio_1 = 0.5
stage_ratio_2 = 0.4
stage_ratio_3 = 0.6
stage_ratio_4 = un_prune

target_pruning_ratio = {
    # stage 1
    'backbone.conv1_(0, 64)_64': un_prune,  # short cut layers
    'backbone.layer1.0.conv1_(0, 64)_64': stage_ratio_1,
    'backbone.layer1.1.conv1_(0, 64)_64': stage_ratio_1,
    'backbone.layer1.2.conv1_(0, 64)_64': un_prune,
    # stage 2
    'backbone.layer2.0.conv1_(0, 128)_128': un_prune,
    'backbone.layer2.0.conv2_(0, 128)_128': un_prune,  # short cut layers
    'backbone.layer2.1.conv1_(0, 128)_128': stage_ratio_2,
    'backbone.layer2.2.conv1_(0, 128)_128': stage_ratio_2,
   

### 5. 运行

In [12]:
! timeout 2 python ./tools/train.py $prune_config_path

# 自动生成剪枝Config

我们提供了一键生成剪枝config的工具get_prune_config.py

In [13]:
! python ./tools/pruning/get_l1_prune_config.py -h

usage: get_l1_prune_config.py [-h] [--checkpoint CHECKPOINT] [--subnet SUBNET]
                              [-o O]
                              config

Get the config to prune a model.

positional arguments:
  config                config of the model

optional arguments:
  -h, --help            show this help message and exit
  --checkpoint CHECKPOINT
                        checkpoint path of the model
  --subnet SUBNET       pruning structure for the model
  -o O                  output path to store the pruning config.


In [14]:
! python ./tools/pruning/get_l1_prune_config.py $work_dir/pretrain.py  --checkpoint $checkpoint_path  -o $prune_config_path   &> /dev/null
! cat $prune_config_path

model = dict(
    _scope_='mmrazor',
    type='ItePruneAlgorithm',
    architecture=dict(
        type='ImageClassifier',
        backbone=dict(
            type='ResNet',
            depth=34,
            num_stages=4,
            out_indices=(3, ),
            style='pytorch'),
        neck=dict(type='GlobalAveragePooling'),
        head=dict(
            type='LinearClsHead',
            num_classes=1000,
            in_channels=512,
            loss=dict(type='CrossEntropyLoss', loss_weight=1.0),
            topk=(1, 5)),
        _scope_='mmcls',
        init_cfg=dict(
            type='Pretrained',
            checkpoint=
            'https://download.openmmlab.com/mmclassification/v0/resnet/resnet34_8xb32_in1k_20210831-f257d4e6.pth'
        ),
        data_preprocessor=dict(
            mean=[123.675, 116.28, 103.53],
            std=[58.395, 57.12, 57.375],
            to_rgb=True)),
    target_pruning_ratio=dict({
        'backbone.conv1_(0, 64)_64': 1.0,
        'backbone.laye

In [15]:
# 清理临时文件
! rm -r $work_dir